This notebook demonstrates another way of generating node embeddings, Graph Convolutional Neural Network, as a comparison to GraphSAGE. The steps of **Spatial Graph Construction** and **Canonical-correlation Analysis-based Embedding generation** and **K-Mean clustering** can be found in file *Step1-GeoAIGeodemographicClassification-SpatialGraphConstruction.ipynb* and *Step3-GeoAIGeodemographicClassification.ipynb*. 

**GCN-based embedding generation**: This demo demonstrates how to generate node embeddings using GCN based on the Deep Graph Infomax algorithm. Deep Graph Infomax is a procedure for training graph machine learning models without supervision. In our case presented here, it helps to convert GCN which is commonly a supervised algorthim into an unsupervised learning process to generate node representations. 

In [ ]:
from stellargraph.mapper import (
    CorruptedGenerator,
    FullBatchNodeGenerator,
    GraphSAGENodeGenerator,
    HinSAGENodeGenerator,
    ClusterNodeGenerator,
)
from stellargraph import StellarGraph
from stellargraph.layer import GCN, DeepGraphInfomax
import stellargraph as sg
from stellargraph import datasets
from stellargraph.utils import plot_history
import networkx as nx

import pandas as pd
from matplotlib import pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from IPython.display import display, HTML

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras import Model

Reading neighbouring information from the saved csv file in *GeoAIGeodemographicClassification-Step1.ipynb*, and converting the neighbouring information into a graph.



In [ ]:
#Read neighbouring information from the saved csv file
#Specify where the csv file is located
edgelist = pd.read_csv('Data/Output/Spatial-Graph/SpatialGraphs.csv')
Gnx = nx.from_pandas_edgelist(edgelist, edge_attr="neighbouring")

nx.set_node_attributes(Gnx, "oa", "neighbouring")

#Read heading information from the csv file
colums = pd.read_csv('Data/Input/Census-Data/GreaterLondon_2011_OAC_Raw_uVariables--zscores.csv', nrows=1).columns.tolist()

graph_col = colums[1:]
print(graph_col)
cols = graph_col+['subject']

#Read node values 
node_data = pd.read_csv('Data/Input/Census-Data/GreaterLondon_2011_OAC_Raw_uVariables--zscores.csv',  sep=',', header=None, names=cols)

print(node_data)

#Assign z-scored census data to the nodes in the graph
node_features = node_data[graph_col]
print(node_features)
#
G = sg.StellarGraph(Gnx, node_features=node_features)

print(G.info())
nodes = list(G.nodes())

Load GCN to take the constructed graph and using *CorruptedGenerator()* function to create the data generators. For details about *CorruptedGenerator()*, please refers to the offical document of [StellarGraph](https://stellargraph.readthedocs.io/en/stable/demos/node-classification/gcn-deep-graph-infomax-fine-tuning-node-classification.html). In a nutshell, *CorruptedGenerator()* returns shuffled node features along with the regular node features and we train our model to discriminate between the two.

In [ ]:
fullbatch_generator = FullBatchNodeGenerator(G, sparse=False)
gcn_model = GCN(layer_sizes=[128], activations=["relu"], generator=fullbatch_generator)

corrupted_generator = CorruptedGenerator(fullbatch_generator)
gen = corrupted_generator.flow(G.nodes())

Using GCN (local pooling) filters...


Load Deep Graph Infomax to take GCN model and compile the model for "training" process. Note that "training" here is not the trainig in supervised algorithms. In such an unsupervised algorithm which does not take labels into account, we train our model to discriminate between the shuffled node features and regular node features.

In [ ]:
infomax = DeepGraphInfomax(gcn_model, corrupted_generator)
x_in, x_out = infomax.in_out_tensors()

model = Model(inputs=x_in, outputs=x_out)
model.compile(loss=tf.nn.sigmoid_cross_entropy_with_logits, optimizer=Adam(lr=1e-3))

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Train the model.

In [ ]:
es = EarlyStopping(monitor="loss", min_delta=0, patience=20)
history = model.fit(gen, epochs=100, verbose=1, callbacks=[es])

Epoch 1/100
1/1 [==============================] - 5s 5s/step - loss: 0.9913
Epoch 2/100
1/1 [==============================] - 2s 2s/step - loss: 0.7999
Epoch 3/100
1/1 [==============================] - 1s 1s/step - loss: 0.7869
Epoch 4/100
1/1 [==============================] - 1s 1s/step - loss: 0.7280
Epoch 5/100
1/1 [==============================] - 1s 1s/step - loss: 0.6667
Epoch 6/100
1/1 [==============================] - 1s 1s/step - loss: 0.6297
Epoch 7/100
1/1 [==============================] - 1s 1s/step - loss: 0.6225
Epoch 8/100
1/1 [==============================] - 1s 1s/step - loss: 0.6076
Epoch 9/100
1/1 [==============================] - 1s 1s/step - loss: 0.5731
Epoch 10/100
1/1 [==============================] - 1s 1s/step - loss: 0.5466
Epoch 11/100
1/1 [==============================] - 1s 1s/step - loss: 0.5267
Epoch 12/100
1/1 [==============================] - 1s 1s/step - loss: 0.5210
Epoch 13/100
1/1 [==============================] - 1s 1s/step - loss: 0.

Extracting node embeddings

In [ ]:
x_emb_in, x_emb_out = gcn_model.in_out_tensors()

In [ ]:
x_out = tf.squeeze(x_emb_out, axis=0)
emb_model = Model(inputs=x_emb_in, outputs=x_out)
all_embeddings = emb_model.predict(fullbatch_generator.flow(G.nodes()))
all_embeddings